In [1]:
from nltk.corpus import wordnet
import numpy as np
import os
import pandas as pd
from sklearn.metrics import accuracy_score
from tqdm import tqdm
import vsm
import data_loading
import nlu_utils

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
# Load the data
msr = data_loading.MSR()
dev = msr.dev()
gutenberg = msr.train_word_word_cooccurence(window=5, vocab_size=12500, load=True)

Loading existing co-occurence matrix


In [10]:
gutenberg.head()

,the,and,of,to,gonna,in,i,he,was,have,...,tinkling,jill,twentysix,disappoint,catches,maturity,candlestick,poll,loathing,lyman
the,1150758,1546998,2303638,1025809,546682,857167,381270,431674,567239,459783,...,298,120,136,96,214,202,260,152,130,152
and,1546998,285101,732932,611510,521166,385879,282628,305280,309942,252834,...,176,148,124,82,112,144,146,70,162,76
of,2303638,732932,203382,372685,568559,353957,185758,187258,247891,205648,...,176,28,110,30,102,210,76,58,102,66
to,1025809,611510,372685,180858,335890,209907,333130,281440,245218,282880,...,32,56,38,204,20,88,46,68,50,72
gonna,546682,521166,568559,335890,132512,307890,184868,199682,246220,192138,...,122,28,64,24,84,68,156,78,56,36


In [11]:
# Calculate PPMI matrix
guten_ppmi = vsm.pmi(gutenberg)

In [12]:
guten_ppmi.head()

,the,and,of,to,gonna,in,i,he,was,have,...,tinkling,jill,twentysix,disappoint,catches,maturity,candlestick,poll,loathing,lyman
the,0.000000,0.089052,0.642524,0.0,0.000000,0.267803,0.000000,0.000000,0.044006,0.00000,...,0.657268,0.00000,0.000000,0.000000,0.307356,0.222857,0.487429,0.000000,0.000000,0.015371
and,0.089052,0.000000,0.009438,0.0,0.004493,0.000000,0.000000,0.000000,0.000000,0.00000,...,0.642765,0.48543,0.314137,0.000000,0.171985,0.396509,0.422460,0.000000,0.520352,0.000000
of,0.642524,0.009438,0.000000,0.0,0.246828,0.050750,0.000000,0.000000,0.000000,0.00000,...,0.798063,0.00000,0.349635,0.000000,0.233757,0.929101,0.000000,0.000000,0.213027,0.000000
to,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.093389,0.056319,0.000000,0.11207,...,0.000000,0.00000,0.000000,1.017862,0.000000,0.156475,0.000000,0.000000,0.000000,0.032705
gonna,0.000000,0.004493,0.246828,0.0,0.000000,0.247356,0.000000,0.000000,0.212895,0.00000,...,0.767640,0.00000,0.144077,0.000000,0.375641,0.137541,0.980048,0.323495,0.000000,0.000000


In [13]:
class PPMIModel:
    def __init__(self, corpus_pmi, try_synonyms=True, verbose=False):
        self.corpus_pmi = corpus_pmi
        self.index_to_label = ['a', 'b', 'c', 'd', 'e']
        self.try_synonyms = try_synonyms
        self.verbose = verbose
    
    def answer(self, problem, try_synonyms=True):
        scores = []
        question = problem['question']
        scores.append(self.score(question, problem['a)'])) 
        scores.append(self.score(question, problem['b)'])) 
        scores.append(self.score(question, problem['c)'])) 
        scores.append(self.score(question, problem['d)'])) 
        scores.append(self.score(question, problem['e)']))
        return self.index_to_label[np.argmax(scores)], scores
    
    def approx_ppmi(self, proposal_token, proposal_synonyms, word_token):
        pos = nlu_utils.spacy_to_wn_tag(word_token.pos_)
        word_synonyms = nlu_utils.get_alternate_words(word_token.norm_, pos)
        # try matching using different versions of the proposal word
        for psyn in proposal_synonyms:
            score = self.ppmi(psyn, word_token.norm_)
            if score is not None:
                if self.verbose:
                    print("Used synonym for proposal word: {} -> {}".format(proposal_token.text, psyn))
                return score
        # try matching using different versions of the non-proposal word
        for wsyn in word_synonyms:
            score = self.ppmi(wsyn, proposal_token.norm_)
            if score is not None:
                if self.verbose:
                    print("Used synonym: {} -> {}".format(word_token.text, wsyn))
                return score
        # Next just try all combos
        for psyn in proposal_synonyms:
            for wsyn in word_synonyms:
                score = self.ppmi(psyn, word_token.norm_)
                if score is not None:
                    if self.verbose:
                        print("Used synonym: {} -> {} and {} -> {}".format(proposal_token.text, psyn, word_token.text, wsyn))
                    return score
        if self.verbose:
            print("UNABLE TO FIND ANY SYNONYMS IN VOCABULARY")
        return None

    def ppmi(self, proposal, word):
        try:
            return self.corpus_pmi.loc[proposal, word]
        except KeyError:
            return None

    def substitute(self, sentence, proposal):
        sentence_list = sentence.split()
        i = sentence_list.index('_____')
        sentence_list[i] = proposal
        return ' '.join(sentence_list)

    def score(self, sentence, proposal):
        full_sentence = self.substitute(sentence, proposal)
        doc = nlu_utils.get_spacy_doc(full_sentence)
        _, proposal_token = nlu_utils.get_token(doc, proposal)

        if self.try_synonyms:
            pos = nlu_utils.spacy_to_wn_tag(proposal_token.pos_)
            synonyms = nlu_utils.get_alternate_words(proposal_token.norm_, pos)

        tot_score = 0
        for token in doc:
            if token == proposal_token:  # !!! This is dubious (might be 'is', not ==)
                continue
            if token.is_punct or token.is_space:
                continue
            score = self.ppmi(proposal_token.norm_, token.norm_)
            if score is None and self.try_synonyms:
                score = self.approx_ppmi(proposal_token, synonyms, token)
            tot_score += score if score is not None else 0
        return tot_score

In [14]:
model = PPMIModel(guten_ppmi, try_synonyms=True, verbose=False)
print("Making predictions")
predictions = []
scores = []
for i, (_, problem) in enumerate(dev.iterrows()):
    ans, sc = model.answer(problem)
    predictions.append(ans)
    scores.append(sc)
    if i % 25 == 0:
        print("------------------------------------------------------------------------------")
        print(problem['question'])
        print(problem)
        print(ans)
        print(sc)
        

Making predictions
------------------------------------------------------------------------------
His hair and whiskers were shot with gray , and his face was all crinkled and _____ like a withered apple.
question    His hair and whiskers were shot with gray , an...
a)                                                 chattering
b)                                                picturesque
c)                                                    hopeful
d)                                                   puckered
e)                                                 glistening
answer                                                      d
Name: 97, dtype: object
e
[7.540923348761015, 3.1314636414005577, 2.8518341539149037, 9.016250098891543, 21.4926366773329]
------------------------------------------------------------------------------
He has been very _____ to us , and hardly a day has passed that he has not called at the Hall to see how we were getting on.
question    He has been very _____

------------------------------------------------------------------------------
Perhaps our _____ now may do something to make it less obscure.
question    Perhaps our _____ now may do something to make...
a)                                                       gate
b)                                                    weather
c)                                                      visit
d)                                                      limbs
e)                                                     skirts
answer                                                      c
Name: 68, dtype: object
c
[0.0906528642199345, 0.5820622776315083, 2.734961211573895, 0.30973877447065484, 0.0]
------------------------------------------------------------------------------
Left his _____ at ten o'clock at night , and has not been heard of since.
question    Left his _____ at ten o'clock at night , and h...
a)                                                     defeat
b)                                

------------------------------------------------------------------------------
They were all three standing in a _____ in front of the altar.
question    They were all three standing in a _____ in fro...
a)                                                  monastery
b)                                               drawing-room
c)                                                       knot
d)                                                   fortress
e)                                                       nest
answer                                                      c
Name: 157, dtype: object
c
[4.872284569401126, 3.741388764580729, 5.63900642746396, 4.928846051058556, 3.1532033678610216]


In [15]:
print(accuracy_score(dev.loc[:, 'answer'], predictions))

0.5464743589743589


## Results
* BASELINE: Window=5, Vocab=10000, synonyms=True: 0.482371794872
* Window=5, Vocab=10000, synonyms_POS=True, spacy.norm_=true: 0.5336538461538461
* Window=5, Vocab=10000, synonyms_POS=True, spacy.norm_=true: 0.5464743589743589

In [27]:
def top_2_accuracy_score(scores, answers):
    index_to_label = ['a', 'b', 'c', 'd', 'e']
    n_correct = 0
    for i, (_, ans) in enumerate(answers.iteritems()):
        pred1, pred2 = np.argsort(-np.array(scores[i]))[:2]
        pred1 = index_to_label[pred1]
        pred2 = index_to_label[pred2]
        if pred1 == ans or pred2 == ans:
            n_correct += 1
    return n_correct / len(answers)
        
top_2_accuracy_score(scores, dev.loc[:, 'answer'])

0.6907051282051282

In [28]:
def examine_model(predictions, scores):
    for i, (_, problem) in enumerate(dev.iterrows()):
        ans_to_index = {'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4}
        total_diff = 0.0
        total_incorrect = 0
        if predictions[i] != problem['answer']:
            print(problem['question'])
            doc = nlu_utils.get_spacy_doc(problem['question'])
            print(' '.join([t.norm_ for t in doc]))
            print(problem)
            print(predictions[i])
            print(scores[i])
            total_incorrect += 1
            total_diff += (scores[i][ans_to_index[predictions[i]]] - scores[i][ans_to_index[problem['answer']]])
    return total_diff / total_incorrect
        
        
examine_model(predictions, scores)

His hair and whiskers were shot with gray , and his face was all crinkled and _____ like a withered apple.
his hair and whiskers were shot with gray , and his face was all crinkled and _ _ _ _ _ like gonna withered apple .
question    His hair and whiskers were shot with gray , an...
a)                                                 chattering
b)                                                picturesque
c)                                                    hopeful
d)                                                   puckered
e)                                                 glistening
answer                                                      d
Name: 97, dtype: object
e
[8.41877136732423, 4.067458034711444, 3.382770316212859, 1.373420578134755, 15.855537113552309]
It was only after a painful and prolonged scene that she was _____ by the butler and the footman.
it was only after gonna painful and prolonged scene that she was _ _ _ _ _ by the butler and the footman .
question    It w

as we passed it holmes , to my unutterable astonishment , leaned over in front of me and deliberately _ _ _ _ _ the whole thing over .
question    As we passed it Holmes , to my unutterable ast...
a)                                                    spurned
b)                                                  renounced
c)                                                    knocked
d)                                                   accepted
e)                                                    crossed
answer                                                      c
Name: 466, dtype: object
e
[1.608536089062551, 4.342880374037633, 8.041179978122234, 4.324297700242667, 10.893974489074076]
I had come to believe that he was an _____ with no relatives living , but one day , to my very great surprise , he began to talk to me about his brother.
i have come to believe that he was an _ _ _ _ _ with no relatives living , but one day , to my very great surprise , he began to talk to me about his bro

Name: 785, dtype: object
b
[8.92847260531291, 10.933147415683642, 10.053178806864425, 2.7015029069848837, 10.888936169083657]
When he reached the _____ I saw the ragged uncouth figure outlined for an instant against the cold blue sky.
when he reached the _ _ _ _ _ i saw the ragged uncouth figure outlined for an instant against the cold blue sky .
question    When he reached the _____ I saw the ragged unc...
a)                                                    picture
b)                                                       star
c)                                                       pipe
d)                                                      crest
e)                                                    curtain
answer                                                      d
Name: 958, dtype: object
b
[7.027265657309194, 9.315347874504123, 3.885486801961779, 9.184396434142796, 5.0783996819860935]
I was instantly _____ , and , with the two footmen , started off at once in search of the mis

far away we could hear the deep _ _ _ _ _ of the parish clock , which boomed out every quarter of an hour .
question    Far away we could hear the deep _____ of the p...
a)                                                  stillness
b)                                                     horror
c)                                                      tones
d)                                                     shadow
e)                                                  breathing
answer                                                      c
Name: 231, dtype: object
a
[9.228778800959521, 5.564637889044114, 7.481175360555931, 7.402047156172989, 6.969589948317591]
Holmes had brought up a long thin _____ , and this he placed upon the bed beside him.
holmes have brought up gonna long thin _ _ _ _ _ , and this he placed upon the bed beside him .
question    Holmes had brought up a long thin _____ , and ...
a)                                                      hedge
b)                           

my wife have already gone upstairs , and the sound of the _ _ _ _ _ of the hall door some time before told me that the servants have also retired .
question    My wife had already gone upstairs , and the so...
a)                                                  discovery
b)                                                      irony
c)                                                    locking
d)                                                   illusion
e)                                                   teaching
answer                                                      c
Name: 553, dtype: object
a
[7.474048816359854, 4.225343297676248, 7.457260851509952, 2.7940859142208785, 4.557701164559088]
I have a few thousands to invest , d'ye see , and I think I'll _____ them in you.
i have gonna few thousands to invest , d'ye see , and i think i will _ _ _ _ _ them in you .
question    I have a few thousands to invest , d'ye see , ...
a)                                                      d

...

i found the black tor upon which i have seen the solitary _ _ _ _ _ , and from its craggy summit i looked out myself across the melancholy downs .
question    I found the black tor upon which I had seen th...
a)                                                confinement
b)                                                 meditation
c)                                                    watcher
d)                                                   carriers
e)                                                     island
answer                                                      c
Name: 20, dtype: object
e
[3.8800933471569894, 8.922444934852667, 4.89661164114063, 6.458878585261589, 10.849323080409155]
He is as brave as a _____ and as tenacious as a lobster if he gets his claws upon anyone.
he is as brave as gonna _ _ _ _ _ and as tenacious as gonna lobster if he gets his claws upon anyone .
question    He is as brave as a _____ and as tenacious as ...
a)                                       

you must find your own ink , pens , and blotting - paper , but we _ _ _ _ _ this table and chair .
question    You must find your own ink , pens , and blotti...
a)                                                     behold
b)                                                   realised
c)                                                    provide
d)                                                       lose
e)                                                        owe
answer                                                      c
Name: 65, dtype: object
e
[4.465878294189278, 1.6628727707181932, 3.387911910252895, 5.063438133460969, 6.664582674110505]
He is a man of no physical courage , as they are well aware from their _____ the other night.
he is gonna man of no physical courage , as they are well aware from their _ _ _ _ _ the other night .
question    He is a man of no physical courage , as they a...
a)                                                   surfaces
b)                     

i fainted when it was done , and i think that i must have been _ _ _ _ _ for gonna long time .
question    I fainted when it was done , and I think that ...
a)                                                  preparing
b)                                                  senseless
c)                                                   watching
d)                                                     spoilt
e)                                                   mistaken
answer                                                      b
Name: 436, dtype: object
e
[4.943512838197249, 1.8379781461611795, 4.411893357212237, 1.312494703297511, 9.153358506663865]
I have established a right of way through the _____ of old Middleton's park , slap across it , sir , within a hundred yards of his own front door.
i have established gonna right of way through the _ _ _ _ _ of old middleton 's park , slap across it , sir , within gonna hundred yards of his own front door .
question    I have established a right 

we should not have troubled you , only that our friend who speaks greek and who began these _ _ _ _ _ has been forced to return to the east .
question    We should not have troubled you , only that ou...
a)                                                individuals
b)                                               negotiations
c)                                                   outcasts
d)                                                      spurs
e)                                                 sentiments
answer                                                      b
Name: 203, dtype: object
c
[6.8317010035813235, 6.390556821727956, 10.343977348232647, 3.1979044981164195, 7.2735929215243]
If I could get his _____ it might save me a long and weary hunt.
if i could get his _ _ _ _ _ it might save me gonna long and weary hunt .
question    If I could get his _____ it might save me a lo...
a)                                                       meal
b)                                   

1.0438223108629612